In [1]:
# APP 的名稱, 可自由更改
title = "狗狗辨識器"
# APP 的說明, 可自由更改
description = "請輸入一張狗狗照片, 我會告訴你這是什麼品種的狗!"

num_class = 4
table_class_en2ch = {
    "Bichon": "比熊",
    "Maltese": "馬爾濟斯",
    "Poodle": "貴賓犬",
    "Pomeranian": "博美"
}
table_class_en2num = {
    "Bichon": 0,
    "Maltese": 1,
    "Poodle": 2,
    "Pomeranian": 3
}
table_class_num2en = {
    0: "Bichon",
    1: "Maltese",
    2: "Poodle",
    3: "Pomeranian"
}

In [2]:
# install packages
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.3 MB/s eta 

In [3]:
# import build-in libraris
import os
import zipfile

# import 3rd-party libraris
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL.Image as Image

%matplotlib inline

In [15]:
# clone dataset to local disk
!git clone https://github.com/jill1209/doggy_classification.git

# unzip file
import zipfile
with zipfile.ZipFile("/content/doggy_classification/dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/doggy_classification/")

Cloning into 'doggy_classification'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 48 (delta 0), reused 6 (delta 0), pack-reused 41
Receiving objects: 100% (48/48), 66.58 MiB | 35.92 MiB/s, done.


In [16]:
dataset_root = "/content/doggy_classification/dataset"
os.listdir(dataset_root)

['Bichon 3.jpg',
 'Pomeranian 5.jpg',
 'Maltese 7.jpg',
 'Bichon 6.jpg',
 'Bichon 7.jpg',
 'Bichon 10.jpg',
 'Maltese 5.jpg',
 'Maltese 8.jpg',
 'Bichon 5.jpg',
 'Pomeranian 1.png',
 'Pomeranian 6.jpg',
 'Maltese 1.png',
 ' Poodle 9.jpg',
 ' Poodle 6.jpg',
 'Pomeranian 7.jpg',
 ' Poodle 7.jpg',
 ' Poodle 2.jpg',
 'Pomeranian 9.jpg',
 'Pomeranian 3.jpg',
 ' Poodle 4.jpg',
 'Pomeranian 4.jpg',
 'Maltese 3.jpg',
 'Maltese 2.png',
 ' Poodle 5.jpg',
 'Bichon 8.jpg',
 'Pomeranian 2.png',
 'Maltese 6.jpg',
 'Maltese 4.jpg',
 'Bichon 4.jpg',
 ' Poodle 10.jpg',
 'Bichon 9.jpg',
 'Pomeranian 8.jpg',
 ' Poodle 1.jpg',
 ' Poodle 3.jpg',
 'Bichon 1.jpg',
 ' Poodle 8.jpg',
 'Bichon 2.jpg']

In [17]:
data = []
target = []

for file_name in os.listdir(dataset_root):
    img_path = os.path.join(dataset_root, file_name)
    img = load_img(img_path, target_size=(224,224))
    x = img_to_array(img)
    y = table_class_en2num[file_name.strip().split('.')[0].split(' ')[0]]
    data.append(x)
    target.append(y)

In [18]:
data = np.array(data)
x_train = preprocess_input(data)
y_train = to_categorical(target, num_class)  # 做 one-hot enconding。

### 3. 用 ResNet50 打造我們的神經網路

ResNet50 是 2015 ImageNet 的冠軍, 我們用第二版來試試。原本 ImageNet 是做了 1,000 個類別的圖形辨識。我們想直接用來辨識八哥, 就是把最後一層 (通常就 1,000 個輸出的 dense 層) 砍掉 (`include_top=False`), 然後換我們的就好。

再來我們可以把每個 filter 的結果做個大總合, 例如算每個 filter 計分板的總平均 (global average pooling), 這本來該我們自己做, 但是 `tf.Keras` 是善良無比的幫我們做好。只要下個參數 `pooling="avg"`)。

In [19]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
resnet = ResNet50V2(include_top=False, pooling="avg")

94668760/94668760 [==============================] - 0s 0us/step


再來就是正式打造我們遷移學習版的函數學習機! 可以發現我們只是加入了最後一層...

In [21]:
model = Sequential()
model.add(resnet)
model.add(Dense(num_class, activation='softmax'))

# 我們是遷移式學習, 原本 ResNet 的部份我們當然沒有重新訓練的意思。於是就設這邊不需要訓練。
resnet.trainable = False

# 欣賞我們的神經網路
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 4)                 8196      
                                                                 
Total params: 23572996 (89.92 MB)
Trainable params: 8196 (32.02 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [22]:
# 組裝我們的函數學習機
# 這裡我們用分類時非常標準的 categorical_crossentropy, 順便試試有名的 adam 學習法。
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
# 訓練 fit
# model.fit(x_train, y_train, batch_size=10, epochs=10)
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
2/2 [==============================] - 11s 722ms/step - loss: 1.5742 - accuracy: 0.4054
Epoch 2/10
2/2 [==============================] - 7s 718ms/step - loss: 1.1098 - accuracy: 0.4865
Epoch 3/10
2/2 [==============================] - 5s 662ms/step - loss: 0.8857 - accuracy: 0.6216
Epoch 4/10
2/2 [==============================] - 7s 718ms/step - loss: 0.7076 - accuracy: 0.7838
Epoch 5/10
2/2 [==============================] - 5s 730ms/step - loss: 0.5679 - accuracy: 0.8378
Epoch 6/10
2/2 [==============================] - 7s 1s/step - loss: 0.4495 - accuracy: 0.8919
Epoch 7/10
2/2 [==============================] - 5s 671ms/step - loss: 0.3722 - accuracy: 0.9189
Epoch 8/10
2/2 [==============================] - 6s 1s/step - loss: 0.3130 - accuracy: 0.9189
Epoch 9/10
2/2 [==============================] - 6s 700ms/step - loss: 0.2551 - accuracy: 0.9459
Epoch 10/10
2/2 [==============================] - 6s 663ms/step - loss: 0.2101 - accuracy: 0.9730


In [24]:
# 預測
# 我們先用 model.evaluate 看一下模型表現得如何
loss, acc = model.evaluate(x_train, y_train)
print(f"Loss: {loss}")
print(f"Accuracy: {acc}")

y_predict = np.argmax(model.predict(x_train), -1)
print(y_predict)
print(target)

2/2 [==============================] - 8s 700ms/step - loss: 0.1869 - accuracy: 1.0000
Loss: 0.1869032084941864
Accuracy: 1.0
2/2 [==============================] - 7s 684ms/step
[0 3 1 0 0 0 1 1 0 3 3 1 2 2 3 2 2 3 3 2 3 1 1 2 0 3 1 1 0 2 0 3 2 2 0 2 0]
[0, 3, 1, 0, 0, 0, 1, 1, 0, 3, 3, 1, 2, 2, 3, 2, 2, 3, 3, 2, 3, 1, 1, 2, 0, 3, 1, 1, 0, 2, 0, 3, 2, 2, 0, 2, 0]


### 6. 用 `gradio` 打造八哥辨識 web app!

In [25]:
def resize_image(inp):
    # 將 NumPy array 轉換成 PIL Image 對象
    img = Image.fromarray(inp)

    # 將圖片調整為 224x224 像素
    img_resized = img.resize((224, 224), Image.Resampling.LANCZOS)

    # 將調整大小後的圖片轉換回 NumPy array
    img_array = np.array(img_resized)

    return img_array

In [26]:
def classify_image(inp):
    img_array = resize_image(inp)
    inp = img_array.reshape((1, 224, 224, 3))
    inp = preprocess_input(inp)
    prediction = model.predict(inp).flatten()
    return {table_class_en2ch[table_class_num2en[i]]: float(prediction[i]) for i in range(num_class)}

In [27]:
image = gr.Image(label="狗狗照片")
label = gr.Label(num_top_classes=num_class, label="AI辨識結果")

In [28]:
# 我們將狗狗數據庫中的圖片拿出來當作範例圖片讓使用者使用
sample_images = []
for file_name in os.listdir(dataset_root):
    # thedir = base_dir + categories[i]
    # for fname in os.listdir(thedir):
    sample_images.append(os.path.join(dataset_root, file_name))

最後，將所有東西組裝在一起，就大功告成了！

In [ ]:
gr.Interface(fn=classify_image,
             inputs=image,
             outputs=label,
             title=title,
             description=description,
             examples=sample_images,
             examples_per_page=len(os.listdir(dataset_root))).launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://aaf3e2b058804b9fa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
